In [2]:
%logstop
%logstart -rtq ~/.logs/pw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [4]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

mkdir: cannot create directory ‘pw-data’: File exists
File ‘./pw-data/201701scripts_sample.json.gz’ already there; not retrieving.

File ‘./pw-data/practices.json.gz’ already there; not retrieving.



## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [5]:
import gzip
import simplejson as json

In [6]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [7]:
scripts[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500}]

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [8]:
practices[:2]

[{'code': 'A81001',
  'name': 'THE DENSHAM SURGERY',
  'addr_1': 'THE HEALTH CENTRE',
  'addr_2': 'LAWSON STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 1HU'},
 {'code': 'A81002',
  'name': 'QUEENS PARK MEDICAL CENTRE',
  'addr_1': 'QUEENS PARK MEDICAL CTR',
  'addr_2': 'FARRER STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 2AW'}]

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [9]:
sum([script['items'] for script in scripts])

4410054

In [10]:

def describe(key):

    total = 0
    avg = 0
    s = 0
    q25 = 0
    med = 0
    q75 = 0
    
    medlist = []
    
    for i in scripts:
        medlist.append(i[key])
    total = sum(medlist)
    
    medlist.sort()
    size = len(medlist)
    
    avg = total / size
    
    sample = len(scripts) - 1
    
    medlist_sum =[(scripts[i][key] - avg) **2 for i  in range (len(scripts))]
    s = (sum(medlist_sum) / sample) ** 0.5
    
    a = len(medlist) // 2
    
    if len(medlist) % 2:
        med = (medlist[a] + medlist[a-1]) / 2
        q25 = medlist [int(size * 0.25) - 1]
        q75 = medlist[int(size * 0.75) - 1]
    
    else:
        med = medlist[a]
        q25 = medlist[int(size*0.25)-1]
        q75 = medlist[int(size*0.75)-1]

    return (total, avg, s, q25, med, q75)

In [11]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]

In [12]:
summary

[('items', (4410054, 11.522744731217633, 33.11220959820685, 1, 3, 8)),
 ('quantity', (316356836, 826.5883059943667, 3872.186073305146, 30, 120, 466)),
 ('nic',
  (29048309.790000338,
   75.89844899484315,
   197.57308474088356,
   7.7,
   22.62,
   65.94)),
 ('act_cost',
  (27053937.599999707,
   70.68748295124895,
   183.26755837716715,
   7.25,
   21.24,
   61.53))]

In [13]:
grader.score.pw__summary_statistics(summary)

Your score:  1.0


## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [14]:
#we ca n also  write it thus
bnf_name = {script['bnf_name'] for script in scripts} # for all uique bnf_name in our data
assert(len(bnf_name)==5619)

In [15]:
#bnf_names = []
#for i in scripts:
 #   bnf_name = i['bnf_name']
  #  if bnf_name in bnf_names:
   #     continue
    #else:
     #   bnf_names.append(bnf_name)
#print(len(bnf_names))

#assert(len(bnf_names) == 5619)

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [16]:
groups = {name: [] for name in bnf_name}
for script in scripts:
    bnf_names = script['bnf_name']
    groups[bnf_names].append(script)

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [17]:
#items_by_bnf_name =[]

#for bnf_name, data in groups.items():
 #   total= sum([x['items'] for x in data])
  #  items_by_bnf_name.append((bnf_name,total))

In [18]:
#items_by_bnf_name[:2]


In [19]:
max_value = 0
max_item = ''
for group in groups:
  #  print(group)
    total_items=0
    for i in groups[group]:
        total_items+=i['items']
#         print(i['items'])
    if total_items>max_value:
        max_value=total_items
        max_item=group


max_item = [(max_item, max_value)]

**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [20]:
grader.score.pw__most_common_item(max_item)

Your score:  1.0


**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [21]:
def group_by_field(data, fields):
    groups = {}
    for d in data:
        key = tuple(d[f] for f in fields)
        if key in groups:
            groups[key].append (d)
        else:
            groups[key] = [d]
    return groups

In [22]:
def sum_values(iterable, key):
    return sum([data[key] for data in iterable])

In [23]:
sum_values(scripts, 'items')

4410054

In [24]:
groups = group_by_field(scripts, ('bnf_name',))
test_max_item = max_item

assert test_max_item == max_item

## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [25]:
practice_postal = {}
for practice in practices:
    if practice['code'] in practice_postal:
        practice_postal[practice['code']] = min(practice['post_code']
                                     , practice_postal[practice['code']])
    else:
        practice_postal[practice['code']] = practice['post_code']


**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [26]:
assert practice_postal['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [27]:
joined = scripts[:] # here, we are creating a copy of our list i.e reads the items from the beginning to the end 
for script in joined:
    script['post_code'] = practice_postal[script['practice']]

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [28]:
grouped = list(group_by_field(joined, ('post_code',)).items())
#items_by_post = group_by_field(joined, ('post_code',))

In [29]:
items_by_post = [(post_code[0], sum(group['items'] for group in groups))
                 for post_code, groups in grouped]

In [30]:
items_by_post = sorted(items_by_post)

In [31]:
#all_postal_totals = []
#for i in range (len(items_by_post)): 
#    all_postal_totals.append(items_by_post[i])

In [32]:
postal_totals = sorted (items_by_post)[:100]

In [33]:
grader.score.pw__postal_totals(postal_totals)

Your score:  1.0


## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [34]:
# find the total items for each bnf name and postal code

grouped_by_bnf_name_and_post_code = group_by_field(joined, ('post_code', 'bnf_name'))

In [1]:
grouped_by_bnf_name_and_post_code[('SK11 6JL', 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F')]

NameError: name 'grouped_by_bnf_name_and_post_code' is not defined

In [36]:
total_items_by_bnf_post = []
for key, val in grouped_by_bnf_name_and_post_code.items():
    total_items_by_bnf_post.append((key, sum_values(val,'items')))
#calculate the sum of all the "items" field in the list called val

In [37]:
# list comprehensions
[(key, sum_values(val, 'items')) for key, val in grouped_by_bnf_name_and_post_code.items()]

[(('SK11 6JL', 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F'), 5),
 (('SK11 6JL', 'Alginate_Raft-Forming Oral Susp S/F'), 3),
 (('SK11 6JL', 'Sod Algin/Pot Bicarb_Susp S/F'), 94),
 (('SK11 6JL', 'Sod Alginate/Pot Bicarb_Tab Chble 500mg'), 9),
 (('SK11 6JL', 'Gaviscon Infant_Sach 2g (Dual Pack) S/F'), 41),
 (('SK11 6JL', 'Gaviscon Advance_Liq (Aniseed) (Reckitt)'), 98),
 (('SK11 6JL', 'Gaviscon Advance_Tab Chble Mint(Reckitt)'), 16),
 (('SK11 6JL', 'Gaviscon Advance_Liq (Peppermint) S/F'), 65),
 (('SK11 6JL', 'Peptac_Liq (Peppermint) S/F'), 14),
 (('SK11 6JL', 'Alverine Cit_Cap 60mg'), 10),
 (('SK11 6JL', 'Hyoscine Butylbrom_Inj 20mg/ml 1ml Amp'), 3),
 (('SK11 6JL', 'Hyoscine Butylbrom_Tab 10mg'), 153),
 (('SK11 6JL', 'Mebeverine HCl_Tab 135mg'), 84),
 (('SK11 6JL', 'Mebeverine HCl_Cap 200mg M/R'), 194),
 (('SK11 6JL', 'Peppermint Oil_Cap E/C 0.2ml'), 42),
 (('SK11 6JL', 'Peppermint Oil_Cap E/C 0.2ml M/R'), 5),
 (('SK11 6JL', 'Colpermin_Cap E/C 0.2ml M/R'), 6),
 (('SK11 6JL', 'Ispag/Mebeverin

In [38]:
total_items_by_bnf_post[:3]

[(('SK11 6JL', 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F'), 5),
 (('SK11 6JL', 'Alginate_Raft-Forming Oral Susp S/F'), 3),
 (('SK11 6JL', 'Sod Algin/Pot Bicarb_Susp S/F'), 94)]

In [39]:
#total_items_by_bnf_post = ...
assert len(total_items_by_bnf_post) == 141196

Let's use `total_items` to find the maximum item total for each postal code. To do this, we will want to regroup `total_items_by_bnf_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_items` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in the resulting `total_items_by_post` after grouping `total_items` by `'post_code'`.

In [40]:
total_items_by_bnf_post[0]

(('SK11 6JL', 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F'), 5)

In [41]:
total_items_by_bnf = [{'post_code':key[0], 'bnf_name': key[1], 'total':val}
                      for key, val in total_items_by_bnf_post]

In [42]:
total_items_by_bnf[0]

{'post_code': 'SK11 6JL',
 'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
 'total': 5}

In [43]:
grouped_by_post_code = group_by_field(total_items_by_bnf, ('post_code',))
total_items_by_post = [(key[0], sum_values(val, 'total'))
                      for key, val in grouped_by_post_code.items()]

In [44]:
#total_items = ...
assert len(total_items_by_post) == 118

In [45]:
total_items_by_post[:2]

[('SK11 6JL', 110071), ('CW5 5NX', 38797)]

In [46]:
total_items_by_post_dict = dict(total_items_by_post)

In [47]:
total_items_by_post_dict['SK11 6JL']

110071

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [48]:
list(grouped_by_post_code.values())[:2]

[[{'post_code': 'SK11 6JL',
   'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
   'total': 5},
  {'post_code': 'SK11 6JL',
   'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
   'total': 3},
  {'post_code': 'SK11 6JL',
   'bnf_name': 'Sod Algin/Pot Bicarb_Susp S/F',
   'total': 94},
  {'post_code': 'SK11 6JL',
   'bnf_name': 'Sod Alginate/Pot Bicarb_Tab Chble 500mg',
   'total': 9},
  {'post_code': 'SK11 6JL',
   'bnf_name': 'Gaviscon Infant_Sach 2g (Dual Pack) S/F',
   'total': 41},
  {'post_code': 'SK11 6JL',
   'bnf_name': 'Gaviscon Advance_Liq (Aniseed) (Reckitt)',
   'total': 98},
  {'post_code': 'SK11 6JL',
   'bnf_name': 'Gaviscon Advance_Tab Chble Mint(Reckitt)',
   'total': 16},
  {'post_code': 'SK11 6JL',
   'bnf_name': 'Gaviscon Advance_Liq (Peppermint) S/F',
   'total': 65},
  {'post_code': 'SK11 6JL',
   'bnf_name': 'Peptac_Liq (Peppermint) S/F',
   'total': 14},
  {'post_code': 'SK11 6JL', 'bnf_name': 'Alverine Cit_Cap 60mg', 'total': 10},
  {'post_code': 'SK11 6JL

In [49]:
max_items_by_post = [sorted(val, key=lambda x: (-x['total'], x['bnf_name']))[0]
                    for val in grouped_by_post_code.values()]

In [50]:
max_items_by_post[:5]

[{'post_code': 'SK11 6JL',
  'bnf_name': 'Omeprazole_Cap E/C 20mg',
  'total': 3219},
 {'post_code': 'CW5 5NX',
  'bnf_name': 'Omeprazole_Cap E/C 20mg',
  'total': 1419},
 {'post_code': 'CW1 3AW',
  'bnf_name': 'Omeprazole_Cap E/C 20mg',
  'total': 2364},
 {'post_code': 'CW7 1AT',
  'bnf_name': 'Omeprazole_Cap E/C 20mg',
  'total': 1655},
 {'post_code': 'CH65 6TG',
  'bnf_name': 'Lansoprazole_Cap 30mg (E/C Gran)',
  'total': 688}]

In [51]:
total_post_by_dict = dict(total_items_by_post)

In [52]:
total_items_by_post[:5]

[('SK11 6JL', 110071),
 ('CW5 5NX', 38797),
 ('CW1 3AW', 64104),
 ('CW7 1AT', 43164),
 ('CH65 6TG', 25090)]

In [53]:
max_items_by_post_all = [(x['post_code'], x['bnf_name'], x['total'] / total_post_by_dict[x['post_code']])
                        for x in max_items_by_post]

In [61]:
#sorted(max_items_by_post_all, key=lambda x:x[0])

In [62]:
items_by_region = sorted(max_items_by_post_all)[:100]

In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that we previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [63]:
#items_by_region = [('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.0341508247)] * 100

In [64]:
grader.score.pw__items_by_region(items_by_region)

Your score:  1.0


*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*